In [1]:
import pandas as pd
import numpy as np
from darts import concatenate, TimeSeries
from darts.dataprocessing.transformers import MinTReconciliator  # noqa
from darts.metrics import mae, rmse, mape  # noqa
from darts.models import (
    LinearRegressionModel,
    NaiveSeasonal,
    ExponentialSmoothing 

)
from darts.utils.model_selection import train_test_split
from darts.utils.utils import ModelMode, SeasonalityMode
from darts.utils.likelihood_models import GaussianLikelihood
import matplotlib.pyplot as plt
from tasks_support_system_ai.utils import get_correct_data_path
from tasks_support_system_ai.readers import read_proper_ts_tree, ts_read_daily_tickets
import seaborn as sns
import random
import matplotlib.cm as cm
from sklearn.ensemble import IsolationForest
import warnings

Пока не работает

In [ ]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from darts import TimeSeries
from tasks_support_system_ai.utils import get_correct_data_path
from tasks_support_system_ai.readers import read_proper_ts_tree, ts_read_daily_tickets
from darts.models import LinearRegressionModel, NaiveSeasonal, ExponentialSmoothing
import random
import warnings

# Кастомные классы для обработки данных
class DataLoader(BaseEstimator, TransformerMixin):
    def __init__(self, data_path):
        self.data_path = data_path

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Загрузка данных
        df = ts_read_daily_tickets(self.data_path)
        tree = read_proper_ts_tree(get_correct_data_path("custom_data/tree_proper.csv"))
        top_level_tree = tree[(tree["level"] == 1) & (tree["full_load"] != 0)]
        top_level_queues = list(
            top_level_tree[top_level_tree["full_load"] >= 1300000]["queueId"].values
        )
        return top_level_queues

class AnomalyDetector(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Для каждого элемента в X (каждая очередь), находим аномалии с помощью IsolationForest
        self.updated_data = {}
        for queue_id in X:
            result = get_df_slice(queue_id)  # Применяем функцию для среза данных
            outliers_fraction = 0.01
            model = IsolationForest(contamination=outliers_fraction, random_state=42)
            result["anomaly"] = model.fit_predict(result[["new_tickets"]])

            # Заменяем аномалии линейной интерполяцией
            result["new_tickets_interp"] = result["new_tickets"].copy()
            result.loc[result["anomaly"] == -1, "new_tickets_interp"] = np.nan
            result["new_tickets_interp"] = result["new_tickets_interp"].interpolate(method="linear")

            self.updated_data[queue_id] = result
        return self

    def transform(self, X):
        # Возвращаем уже обновленные данные
        return self.updated_data

class GridSearchForModels(BaseEstimator, TransformerMixin):
    def __init__(self, models_to_try):
        self.models_to_try = models_to_try

    def fit(self, X, y=None):
        self.model_metrics = {}
        for queue_id, data in X.items():
            # Разделяем на обучающую и тестовую выборку
            ts = TimeSeries.from_dataframe(
                data,
                value_cols="new_tickets_interp",
                fill_missing_dates=True,
                fillna_value=0,
                freq="D",
            )
            X_train, X_test = train_test_split(ts, test_size=0.25)
            model_metrics = {}

            for model_name, model_class in self.models_to_try.items():
                print(f"Analyzing {model_name} for Queue {queue_id}...")

                # Для линейной регрессии
                if model_name == "Regression":
                    optimal_params = grid_search_regression(X_train, X_test)
                    model = model_class(
                        lags=sorted(optimal_params[1]['lags']),
                        output_chunk_length=optimal_params[1]['output_chunk_length'],
                        multi_models=optimal_params[1]['multi_models']
                    )
                elif model_name == 'Naive Seasonal':
                    optimal_params = grid_search_naive_seasonal(X_train, X_test)
                    model = model_class(K=optimal_params[1]['K'])
                elif model_name == 'Exponential Smoothing':
                    optimal_params = grid_search_ES(X_train, X_test)
                    model = model_class(
                        trend=optimal_params[1]['trend'],
                        seasonal=optimal_params[1]['seasonal'],
                        seasonal_periods=optimal_params[1]['seasonal_periods']
                    )
                else:
                    model = model_class

                model.fit(X_train)
                X_pred = model.predict(60)

                RMSE_score = rmse(X_pred, X_test)
                MAE_score = mae(X_pred, X_test)
                MAPE_score = mape(X_pred, X_test)
                model_metrics[model_name] = (RMSE_score, MAE_score, MAPE_score)

            self.model_metrics[queue_id] = model_metrics
        return self

    def transform(self, X):
        # Возвращаем метрики для всех очередей
        return self.model_metrics

# Этап сохранения результатов
class SaveResultsToCsv(BaseEstimator, TransformerMixin):
    def __init__(self, filename="model_metrics.csv"):
        self.filename = filename

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        data = []
        for queue_id, metrics in X.items():
            for model_name, (rmse, mae, mape) in metrics.items():
                data.append({
                    "Queue ID": queue_id,
                    "Model": model_name,
                    "RMSE": rmse,
                    "MAE": mae,
                    "MAPE": mape
                })
        df = pd.DataFrame(data)
        df.to_csv(self.filename, index=False)
        print(f"Results saved to {self.filename}")
        return X

# Строим пайплайн
pipeline = Pipeline([
    ('data_loader', DataLoader(data_path="tickets_daily.csv")),
    ('anomaly_detector', AnomalyDetector()),
    ('grid_search', GridSearchForModels(models_to_try={
        "Regression": LinearRegressionModel,
        "Naive Seasonal": NaiveSeasonal,
        "Exponential Smoothing": ExponentialSmoothing
    })),
    ('save_results', SaveResultsToCsv())
])

# Запускаем пайплайн
results = pipeline.fit_transform(None)  # Мы не передаем данные напрямую, потому что они загружаются в DataLoader


NameError: name 'get_df_slice' is not defined